# Historical Weather Data Using PAIRS API

NOTE: Use this file to fill empty visibility and Temperature values ONLY if you're using the Kaggle dataset here - https://www.kaggle.com/sobhanmoosavi/us-accidents

DO NOT Run this notebook if you're using the dataset we provided here - ***. 
This has the temperature and visibility values filled already.

### Environment Setup

Run `pip install ibmpairs -U ` and ensure the version is 0.1.3 as this has the latest Auth features.

API quickstart guide: https://pairs.res.ibm.com/tutorial/tutorials/api/quickstart.html


Dataset Explorer: https://ibmpairs.mybluemix.net/data-explorer

Login Credentials: 
    * username - ****
    * password - ****


In [7]:
from ibmpairs import paw, authentication
import numpy as np
import pandas as pd
import json
import datetime
from tzwhere import tzwhere
from datetime import datetime   
import pytz
from timezonefinder import TimezoneFinder

In [8]:
PAIRS_SERVER = '****'
PAIRS_API_KEY = '***'
PAIRS_CREDENTIALS = authentication.OAuth2(api_key = PAIRS_API_KEY)

Query parameters of interest:

- layer_id: the number associated with the dataset layer you are interested in. In the data explorer, find a dataset you want to query and click "Data Layers" to get the list of datalayers for that dataset. This is the ID you will use in the query
- coordinates: (latitude, longitude) of the point you want data for
- snapshot: the timestamp of the point you want data for. If data is missing for the specific timestamp, snapshot will grab the closest timestamp that has data

Data from the query is returned in the form of a dataframe with the metadata. The `value` column is the one of interest here, as this shows the data point. The units of the observations can be found in the data explorer window for the specific layer that you queried.

### Values we need to get from API

- Temperature (F)
    - layer 48552 temperature of air 2m above surface (6 hourly)
    - layer 49257 temperature above ground (K) hourly
- wind chill (F)
    - layer 49310 temperature feels like (wind chill in winter and heat index in summer from humidity) in K
- humidity (%)
    - layer 49252 relative humidity of the air (%) hourly
- pressure (in)
    - layer 48544 surface pressure (units are in Pa)
- Visibility (mi)
    - layer 49312 visibility surface (m) but visibility >10 statute miles is 999 0 - hourly
- wind direction
    - layer 50463 wind direction (10 degree intervals where 0 is N, 90 is E, 180 is S, 270 is W) hourly
- wind speed (mph)
    - layer 48657 or 48868 max wind gust speed 10m above surface
    - layer 49313 wind speed (m/s) hourly
- precipitation (in) 
    - layer 26012  hourly recipitation rate
    - layer 48547 snow depth (ECMWF)
    - layer 48866 snowfall (6 hour interval)
    - layer 49249 precipitation past 1 hr (mm)
    - layer 49254 snow past 1 hour (hourly) (m)
- weather condition (text) i.e. fair, cloudy, rainy

- **layer 49314** driving difficulty on a scale of 0 to 100 that takes into account wind, precipitation and fog, hoursly data

## Data Cleaning

In [ ]:

df = pd.read_csv('US_Accidents_Dec20_updated.csv')
df.head()

In [10]:
# fill in missing timezones based on lat/long coordinates
tf = TimezoneFinder()
df['Timezone'] = df.apply(lambda row: tf.timezone_at(lng=row['Start_Lng'],lat=row['Start_Lat']) if pd.isna(row['Timezone']) else row['Timezone'], axis =1)

In [11]:
# standardize timezone names used in pytz package
timezone_map = {'US/Eastern':'America/New_York', 'US/Central':'America/Chicago', 'US/Mountain':'America/Denver', 'US/Pacific': 'America/Los_Angeles'}
df['Timezone'].replace(timezone_map, inplace = True)

In [12]:
# convert a timestamp to UTC using the timezone name
def local_to_utc(local, timestamp):
    local = pytz.timezone(local)
    naive = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    local_dt = local.localize(naive, is_dst=False)
    utc_dt = local_dt.astimezone(pytz.utc)
    utc_str = utc_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    return utc_str


In [14]:
# drop points with timezone not in the US
df = df[df['Timezone'] != 'other']

In [15]:
# drop missing timestamps
df = df[df['Timezone'].notna()]

In [17]:
# remove tailing decimal from timestamp
def clean_timestamp(timestamp):
    if '.' in timestamp:
        return timestamp.split('.')[0]
    else:
        return timestamp
df['Start_Time'] = df['Start_Time'].apply(clean_timestamp)

In [18]:
# create UTC timestamp column using the local timestamp and timezone
df['UTC_timestamp'] =  df.apply(lambda row: local_to_utc(row['Timezone'], row['Start_Time']), axis =1)


## Retrieving Missing Temperature Data Using PAIRS API Queries

In [20]:
# convert a kelvin temperature to fahrenheit
def kelvin_to_fahrenheit(kelvin):
    return ((kelvin - 273.15) * (9/5) + 32)

def m_to_mi(meters):
    # 999 represents > 10 miles so return 10 miles
    if meters == 999:
        return 10
    return (meters/1609.34)

# function to perform a PAIRS query and get the temperature for a given point/timestamp
def get_missing_vals(utc_timestamp, lat, long):
    query_json = {
                'layers': [
                    {'id': 49257},
                    {'id': 49312},
                    {'id': 49314}
                ],
                "spatial" : {'type': 'point', 'coordinates': [lat, long]},
                'temporal' : {'intervals' : [
                    {'snapshot': utc_timestamp}
                ]}
            }

    query = paw.PAIRSQuery(query_json, PAIRS_SERVER, PAIRS_CREDENTIALS, authType = 'api-key')
    query.submit()

    tdf = query.vdf
    temp_k = tdf.iloc[0].value
    visibility_m = tdf.iloc[1].value
    driving_score = tdf.iloc[2].value
    print(temp_k,visibility_m)

    return (kelvin_to_fahrenheit(temp_k), m_to_mi(visibility_m), driving_score)
    
    

In [21]:
def fill_dataframe(missing_df, idx):
    for i in missing_df.index:
        try:
            time = missing_df.at[i, 'UTC_timestamp']
            lat = missing_df.at[i, 'Start_Lat']
            long = missing_df.at[i, 'Start_Lng']
            temp, visibility, driving_score = get_missing_vals(time, lat, long)
            missing_df.at[i, 'Temperature(F)'] = temp
            missing_df.at[i, 'Visibility(mi)'] = visibility
            missing_df.at[i, 'driving_score'] = driving_score
        except Exception as e:
            print("Caught exception : ", e)
            raise AuthException(missing_df, idx)
    return missing_df

In [22]:
# Define Exception class
class AuthException(Exception):
    def __init__(self, df, idx):
        self.df = df
        self.idx = idx

In [25]:
original_df = pd.read_csv('US_Accidents_Dec20_updated.csv')
merged_df = original_df.merge(result_df[['ID', 'Temperature(F)', 'Visibility(mi)']], how = 'left', on = 'ID')

In [26]:
# consolidate merged temperature and visibility data to 1 column
merged_df['Temperature(F)'] = merged_df.apply(lambda row: row['Temperature(F)_y'] if pd.isna(row['Temperature(F)_x']) else row['Temperature(F)_x'], axis =1)
merged_df['Visibility(mi)'] = merged_df.apply(lambda row: row['Visibility(mi)_y'] if pd.isna(row['Visibility(mi)_x']) else row['Visibility(mi)_x'], axis =1)

In [28]:
merged_df = merged_df.drop(['Temperature(F)_x', 'Visibility(mi)_x', 'Visibility(mi)_y', 'Temperature(F)_y','Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'  ], axis =1)

In [29]:
# Rename this file to US_Accidents_Dec20_updated.csv to be used in later stages.
merged_df.to_csv('updated.csv')